In [4]:
import nsfg1
import pandas as pd
import numpy as np
from importlib import reload
reload(nsfg1)

<module 'nsfg1' from 'c:\\Users\\Ayushi Tripathi\\OneDrive\\Documents\\Github\\NSFG-Data-Analysis\\nsfg1.py'>

In [5]:
preg = nsfg1.read_fem_preg()
preg.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,howenddk,nbrnaliv,...,cmintvw,cmlstyr,cmjan3yr,cmjan4yr,cmjan5yr,quarter,phase,intvwyear,totalwgt_lb1,totalwgt_kg
0,60418,1,NaN,NaN,NaN,NaN,5.0,NaN,NaN,1.0,...,NaN,1369,1345,1333,1321,14,1,201,5.2500,2.381358
1,60418,2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,1.0,...,NaN,1369,1345,1333,1321,14,1,201,4.7500,2.154562
2,60418,3,NaN,NaN,NaN,NaN,5.0,NaN,NaN,1.0,...,NaN,1369,1345,1333,1321,14,1,201,5.2500,2.381358
3,60419,1,33.0,1.0,8.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1376,1345,1333,1321,16,1,201,NaN,NaN
4,60420,1,NaN,NaN,NaN,NaN,6.0,NaN,NaN,1.0,...,NaN,1376,1345,1333,1321,16,1,201,8.8125,3.997279


In [6]:
preg.columns

Index(['caseid', 'pregordr', 'howpreg_n', 'howpreg_p', 'moscurrp', 'nowprgdk',
       'pregend1', 'pregend2', 'howenddk', 'nbrnaliv',
       ...
       'cmintvw', 'cmlstyr', 'cmjan3yr', 'cmjan4yr', 'cmjan5yr', 'quarter',
       'phase', 'intvwyear', 'totalwgt_lb1', 'totalwgt_kg'],
      dtype='object', length=280)

In [7]:
preg.columns[1]

'pregordr'

In [8]:
pregordr = preg.pregordr
type(pregordr)

pandas.core.series.Series

In [9]:
pregordr

0       1
1       2
2       3
3       1
4       1
       ..
9353    3
9354    1
9355    2
9356    1
9357    2
Name: pregordr, Length: 9358, dtype: int64

In [10]:
pregordr[0]

1

In [11]:
pregordr[2:5]

2    3
3    1
4    1
Name: pregordr, dtype: int64

In [12]:
preg.outcome.value_counts().sort_index()

outcome
1    6489
2     947
3      86
4    1469
5     118
6     249
Name: count, dtype: int64

In [13]:
preg.birthwgt_lb1.value_counts().sort_index()

birthwgt_lb1
0.0        6
1.0       34
2.0       47
3.0       67
4.0      196
5.0      586
6.0     1666
7.0     2146
8.0     1168
9.0      363
10.0      82
11.0      17
12.0       7
13.0       2
14.0       2
17.0       1
Name: count, dtype: int64

In [14]:
# `read_fem_resp` is a function that returns the respondent DataFrame
resp = nsfg1.read_fem_resp()

# Create a dictionary to map from each respondent's caseid to a list of indices
respondent_indices = {}
for index, row in resp.iterrows():
    caseid = row['caseid']
    if caseid not in respondent_indices:
        respondent_indices[caseid] = []
    respondent_indices[caseid].append(index)

# Now use this dictionary to select the pregnancy outcomes for a single respondent
# let's say you want to get pregnancy outcomes for the respondent with caseid 60418
respondent_caseid = 60418
if respondent_caseid in respondent_indices:
    indices_for_respondent = respondent_indices[respondent_caseid]
    # `preg` is the pregnancy DataFrame
    pregnancy_outcomes_for_respondent = preg.loc[indices_for_respondent]

    # Display the pregnancy outcomes for the selected respondent
    print(pregnancy_outcomes_for_respondent)
else:
    print(f"Caseid {respondent_caseid} not found in the respondent data.")


Length of DataFrame in read_fem_resp: 5699
   caseid  pregordr  howpreg_n  howpreg_p  moscurrp  nowprgdk  pregend1  \
0   60418         1        NaN        NaN       NaN       NaN       5.0   

   pregend2  howenddk  nbrnaliv  ...  cmintvw  cmlstyr  cmjan3yr  cmjan4yr  \
0       NaN       NaN       1.0  ...      NaN     1369      1345      1333   

   cmjan5yr  quarter  phase  intvwyear  totalwgt_lb1  totalwgt_kg  
0      1321       14      1        201          5.25     2.381358  

[1 rows x 280 columns]


Select the `birthord` column, print the value counts, and compare to results published in the Female Pregnancy File Codebook

In [15]:
df = nsfg1.read_fem_preg()
df.birthord.value_counts().sort_index()

birthord
1.0     3067
2.0     2002
3.0      937
4.0      322
5.0      106
6.0       32
7.0       14
8.0        6
9.0        2
10.0       1
Name: count, dtype: int64

Use `isnull` to count the number of nans

In [16]:
df.birthord.isnull().sum()

2869

Select the `prglngth` column, print the value counts, and compare to results published in the [codebook]

In [17]:
# Categorize the weeks into ranges
bins = [0, 13, 26, float('inf')]  # Using float('inf') for "27 weeks or longer"
labels = ['0-13 weeks', '14-26 weeks', '27 weeks or longer']

# Create a new column with the categorized values
df['prglngth_category'] = pd.cut(df['prglngth'], bins=bins, labels=labels, include_lowest=True)

# Print the value counts for the categorized column
print("Categorized Value Counts:")
print(df['prglngth_category'].value_counts())

Categorized Value Counts:
prglngth_category
27 weeks or longer    6585
0-13 weeks            2323
14-26 weeks            450
Name: count, dtype: int64


To compute the mean of a column, you can invoke the `mean` method on a Series.  For example, here is the mean birthweight in pounds:

In [18]:
# Compute the mean birthweight in pounds
mean_birthweight = df['birthwgt_lb1'].mean()

# Print the result
print("Mean Birthweight in Pounds:", mean_birthweight)

Mean Birthweight in Pounds: 6.703286384976526


Create a new column named <tt>totalwgt_kg</tt> that contains birth weight in kilograms.  Compute its mean.  Remember that when you create a new column, you have to use dictionary syntax, not dot notation.

In [19]:
# Compute the mean of the 'totalwgt_kg' column
mean_totalwgt_kg = df['totalwgt_kg'].mean()

# Print the mean birth weight in kilograms
print("Mean Birth Weight in Kilograms:", mean_totalwgt_kg)

Mean Birth Weight in Kilograms: 3.2300184609756095


`nsfg1.py` also provides `read_fem_resp`, which reads the female respondents file and returns a `DataFrame`:

`DataFrame` provides a method `head` that displays the first five rows (Female Respondent Codebook File)

In [20]:
resp.head()

,caseid,rscrninf,rscrage,rscrhisp,rscrrace,age_a,age_r,cmbirth,agescrn,marstat,...,sest,cmintvw,cmlstyr,cmjan3yr,cmjan4yr,cmjan5yr,quarter,phase,intvwyear,intvlngth
0,60418,5,38,5,5,38,38,921,38,2,...,342,1381,1369,1345,1333,1321,14,1,2015,112.7
1,60419,5,37,5,5,37,37,942,37,2,...,318,1388,1376,1345,1333,1321,16,1,2015,80.0
2,60420,5,35,5,4,33,33,981,33,6,...,339,1388,1376,1345,1333,1321,16,1,2015,65.0
3,60423,1,31,5,4,31,31,1010,31,6,...,355,1387,1375,1345,1333,1321,16,1,2015,65.7
4,60426,5,39,5,5,39,39,897,39,1,...,344,1368,1356,1321,1309,1297,9,1,2013,51.6


Select the `ager` column from `resp` and print the value counts.  How old are the youngest and oldest respondents?

In [21]:
ager_column = resp['ager']

# Print value counts
print(ager_column.value_counts())

# Find the youngest and oldest respondents
youngest_age = ager_column.min()
oldest_age = ager_column.max()

print(f"Youngest respondent age: {youngest_age} years")
print(f"Oldest respondent age: {oldest_age} years")

ager
17    239
32    237
25    235
24    231
26    221
30    217
35    210
31    207
27    206
18    205
29    196
22    193
19    192
23    192
34    190
28    189
16    188
15    186
33    184
43    178
21    176
37    174
36    174
44    171
39    165
41    164
20    161
38    150
40    138
42    129
45      1
Name: count, dtype: int64
Youngest respondent age: 15 years
Oldest respondent age: 45 years


We can use the `caseid` to match up rows from `resp` and `preg`.  For example, we can select the row from `resp` for `caseid` 60418 like this:

In [22]:
selected_rows = resp[resp['caseid'] == 60418]
print(selected_rows)

   caseid  rscrninf  rscrage  rscrhisp  rscrrace  age_a  age_r  cmbirth  \
0   60418         5       38         5         5     38     38      921   

   agescrn  marstat  ...  sest  cmintvw  cmlstyr  cmjan3yr  cmjan4yr  \
0       38        2  ...   342     1381     1369      1345      1333   

   cmjan5yr  quarter  phase  intvwyear  intvlngth  
0      1321       14      1       2015      112.7  

[1 rows x 3207 columns]


In [23]:
caseid_exists = (resp['caseid'] == 60418).any()
print(caseid_exists)

True


And we can get the corresponding rows from `preg` like this:

In [24]:
# `resp` and `preg` are DataFrames
selected_rows_resp = resp[resp['caseid'] == 60418]
merged_data = pd.merge(selected_rows_resp, preg, on='caseid')

# `merged_data` contains rows from both DataFrames where the caseid is 60418
print(merged_data)

   caseid  rscrninf  rscrage  rscrhisp  rscrrace  age_a  age_r  cmbirth_x  \
0   60418         5       38         5         5     38     38        921   
1   60418         5       38         5         5     38     38        921   
2   60418         5       38         5         5     38     38        921   

   agescrn_x  marstat  ...  cmintvw_y  cmlstyr_y  cmjan3yr_y  cmjan4yr_y  \
0         38        2  ...        NaN       1369        1345        1333   
1         38        2  ...        NaN       1369        1345        1333   
2         38        2  ...        NaN       1369        1345        1333   

   cmjan5yr_y  quarter_y  phase_y  intvwyear_y  totalwgt_lb1  totalwgt_kg  
0        1321         14        1          201          5.25     2.381358  
1        1321         14        1          201          4.75     2.154562  
2        1321         14        1          201          5.25     2.381358  

[3 rows x 3486 columns]


How old is the respondent with `caseid` 60418?

In [25]:
selected_row_resp = resp[resp['caseid'] == 60418]

# Extract the age from the selected row
respondent_age = selected_row_resp['ager'].values[0]

print(f"The age of the respondent with caseid 60418 is {respondent_age} years.")

The age of the respondent with caseid 60418 is 38 years.


What are the pregnancy lengths for the respondent with `caseid` 60418?

In [26]:
selected_rows_preg = preg[preg['caseid'] == 60418]

# Extract the pregnancy lengths from the selected rows
pregnancy_lengths = selected_rows_preg['prglngth']

print("Pregnancy lengths for the respondent with caseid 60418:")
print(pregnancy_lengths)

Pregnancy lengths for the respondent with caseid 60418:
0    40
1    36
2    36
Name: prglngth, dtype: int64


What was the birthweight of the first baby born to the respondent with `caseid` 60418?

In [27]:
# Assuming `preg` is your pregnancy DataFrame
selected_rows_preg = preg[preg['caseid'] == 60418]

# Assuming the birthweight columns are 'birthwgt_lb1' and 'birthwgt_oz1'
birthweight_lb = selected_rows_preg['birthwgt_lb1'].iloc[0]
birthweight_oz = selected_rows_preg['birthwgt_oz1'].iloc[0]

# Convert ounces to pounds and add to the total weight in pounds
total_weight_lb = birthweight_lb + birthweight_oz / 16

print(f"Total weight of the first baby born to the respondent with caseid 60418: {total_weight_lb} pounds")

Total weight of the first baby born to the respondent with caseid 60418: 5.25 pounds


Prints a list of outcomes for her pregnancies corresponding to respondent 60418

In [28]:
# Define caseid
caseid = 60418

# Use the make_preg_map function from nsfg1 module
preg_map = nsfg1.make_preg_map(df)

# Get indices for the specified caseid
indices = preg_map[caseid]

# Access the 'outcome' column using the indices
df.outcome[indices].values

array([1, 1, 1], dtype=int64)

This data tells the story of a woman who was pregnant three times, each time ending in miscarriage.

In [30]:
# Define caseid
caseid = 60600

# Use the make_preg_map function from nsfg1 module
preg_map = nsfg1.make_preg_map(df)

# Get indices for the specified caseid
indices = preg_map[caseid]

# Access the 'outcome' column using the indices
df.outcome[indices].values

array([5, 1, 1, 1], dtype=int64)

1. The variable PREGEND1 (13-13) in the dataset records the outcome of pregnancies, with each numeric code representing a specific outcome. The value '5' in this variable indicates "Live birth by Cesarean section," denoting that the pregnancy resulted in a live birth delivered through Cesarean section. This information is derived from the associated codebook, which outlines the meanings of each code. The variable is applicable to completed pregnancies, and occurrences marked as 'INAPPLICABLE' suggest instances where the variable is not relevant. This interpretation provides insights into the mode of delivery for pregnancies resulting in live births in the dataset.

2. The value '1' in the variable PREGEND1 (13-13) signifies "Miscarriage." In the context of the dataset, this code indicates that the pregnancy in question ended in a miscarriage. The numeric codes in PREGEND1 correspond to various pregnancy outcomes, and '1' specifically identifies cases where the pregnancy did not progress to a live birth but resulted in a miscarriage instead. This interpretation is based on information provided in the associated codebook, which outlines the meanings of different codes within the variable.

In [31]:
df.birthord 

0       1.0
1       2.0
2       3.0
3       NaN
4       1.0
       ... 
9353    2.0
9354    1.0
9355    2.0
9356    1.0
9357    2.0
Name: birthord, Length: 9358, dtype: float64

In [32]:
live = preg[preg.outcome == 1]
firsts = live[live.birthord == 1]
others = live[live.birthord != 1]

assert len(live) == 9358
assert len(firsts)

AssertionError: 